# スタサプ RPA

## モジュール読み込み

In [1]:
# selenium 4
import math
import time
import os
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.wait import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
# import selenium.webdriver.common.devtools.v114 as devtools
from selenium.webdriver.common.by import By
# .env file load
from dotenv import load_dotenv
load_dotenv()

True

## インスタンスの生成
### 実行

In [2]:
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)

driver.set_window_position(0, 0)

original_window = driver.current_window_handle

### .env 読み込み

In [3]:
service_url = os.getenv("LOGIN_URL")
email = os.getenv("EMAIL_ADDRESS")
password = os.getenv("PASSWORD")

### ログイン

In [4]:
driver.get(service_url)
username_input_element = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[2]/input')
password_input_element = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[3]/span/input')
login_button = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/button[2]/span[2]')

username_input_element.clear()
password_input_element.clear()
username_input_element.send_keys(email)
password_input_element.send_keys(password)
email, password = None, None
login_button.click()


### URL を設定

In [5]:
studysapuri_uri_dict = {
    "homework_active": "https://learn.studysapuri.jp/ja/todos/active",
    "homework_expired": "https://learn.studysapuri.jp/ja/todos/expired"
}


### 課題状況検査

課題の残りの数をカウントする

In [6]:
driver.get(studysapuri_uri_dict["homework_active"])
WebDriverWait(driver, timeout=10).until(
    lambda d: d.find_elements(
        by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
)

try:
    is_active_homework_empty = WebDriverWait(driver, timeout=4).until(
        lambda d: d.find_elements(
            by=By.CSS_SELECTOR, value="div > ul li > button")
    )
except TimeoutException as err:
    is_active_homework_empty = []

# -------------------------------------------------------------------

driver.get(studysapuri_uri_dict["homework_expired"])
WebDriverWait(driver, timeout=4).until(
    lambda d: d.find_elements(
        by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
)

try:
    is_expired_homework_empty = WebDriverWait(driver, timeout=4).until(
        lambda d: d.find_elements(
            by=By.CSS_SELECTOR, value="div > ul li > button")
    )
except TimeoutException as err:
    is_expired_homework_empty = []

残りの課題の数を表示

In [7]:
print(len(is_active_homework_empty),len(is_expired_homework_empty))

0 4


最初の課題を実行

In [8]:
first_taskwork = driver.find_element(
    by=By.CSS_SELECTOR, value='button[class*="TodoCard"]')
first_taskwork.click()

first_todo = driver.find_element(
    by=By.CSS_SELECTOR, value="div > ul li div > ul > li")
first_todo.click()

In [9]:
tab = []
call_count = 0

In [10]:
def quession_automation() -> None:
    """確認問題をランダムに解く
    """
    global call_count
    call_count += 1
    driver.execute_script("""
    var shuffleArray = (array) => {
            const cloneArray = [...array];
            for (let i = cloneArray.length - 1; i >= 0; i--) {
                let rand = Math.floor(Math.random() * (i + 1));
                let tmpStorage = cloneArray[i];
                cloneArray[i] = cloneArray[rand];
                cloneArray[rand] = tmpStorage;
            }
            return cloneArray;
        };

        setTimeout(() => {
            shuffleArray(document.querySelectorAll(
                "[class*=TopicsPage__Main]  button") )
                .forEach(each => each.click());
        },1000)
        setTimeout(()=>{
            document.querySelector('button[class*=RaisedButton]').click();
        },1500)
    """)
    # 待機
    button = WebDriverWait(driver, timeout=10).until(
        lambda d: d.find_element(
            by=By.CSS_SELECTOR, value="[class*=RaisedButton]")
    )

    # ボタンの有効化待ち
    WebDriverWait(driver, timeout=5).until(expected_conditions.element_to_be_clickable(
        (By.CSS_SELECTOR, "[class*=RaisedButton]")
    ))

    # 完全ロード待ち
    WebDriverWait(driver, timeout=20).until(
        expected_conditions.presence_of_all_elements_located)

    if button.text == "再読み込み":
        time.sleep(2)

    while(not driver.current_url.endswith("result")):
        time.sleep(1 + math.sqrt(call_count))
        try:
            quession_automation()
        except:
            print("ERR: Retry...")
    print("Quiz compreat.")


各チャプタのタブを開く

In [12]:
WebDriverWait(driver, timeout=4).until(lambda d: d.find_element(
    by=By.CSS_SELECTOR, value='ul[class*="LessonStepList"] > li > a')
)

after_second_lesston_list = map(lambda element: element.get_attribute("href"), driver.find_elements(
    by=By.CSS_SELECTOR, value='ul[class*="LessonStepList"] > li > a'))

i = 0

try:
    for href in list(after_second_lesston_list):
        driver.switch_to.new_window("tab")
        tab.append(driver.current_window_handle)
        driver.get(href)

        if "questions" in href.split("/"):
            print(href)
            quession_automation()

        # video wait
        WebDriverWait(driver, timeout=4).until(
            lambda d: d.find_element(
                by=By.TAG_NAME, value="video")
        )
        WebDriverWait(driver,timeout=4).until(
            lambda d: d.find_element(
                by=By.CSS_SELECTOR, value="[class*=hugereplaybutton]")            
        )

        driver.execute_script('document.querySelector("video").volume = 0;')
        driver.execute_script(
            'document.querySelector("video").playbackRate = 16;')
        driver.execute_script(
            'document.querySelector("button[class*=bmpui]").click();')
        driver.execute_script('setTimeout(()=>document.querySelector("video").play(), 1000);')
        driver.execute_script('console.log("RUN");')
        time.sleep(1)

except TimeoutException as err:
    print("Timeout Err")
    print(err)
finally:
    driver.switch_to.window(original_window)


https://learn.studysapuri.jp/ja/assignments/646de2622f868b0714676d40/topics/578dd8099c1cfd001d00018d/questions/1
ERR: Retry...
ERR: Retry...
ERR: Retry...
ERR: Retry...
ERR: Retry...
ERR: Retry...
ERR: Retry...
ERR: Retry...
ERR: Retry...
Quiz compreat.
Quiz compreat.
Quiz compreat.
Quiz compreat.
Quiz compreat.
Timeout Err
Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16



In [13]:
driver.switch_to.window(original_window)
for i in range(1, 30+1):
    print(i)
    time.sleep(1)

for t in tab:
    driver.switch_to.window(t)
    driver.close()

driver.switch_to.window(original_window)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [14]:
driver.quit() # 終了